In [21]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import numpy as np
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

df_day = pd.read_csv(config['Data_filename_day'], parse_dates=['Time'], index_col='Time')
df_day.drop(columns=['Volume', 'Change', 'ChangePercent', 'Vwap'], inplace=True)
# df_day['EMA10'] = ta.ema(df_day['Close'])
# df_day['EMA50'] = ta.ema(df_day['Close'], length=50)
# df_day.dropna(inplace=True)
# df_day['MTI'] = (df_day['EMA10'] - df_day['EMA50'])
# mti_green_mask = ((df_day['MTI'] > 0.1) & (df_day['Close'].shift(1) > df_day['Close'].shift(2)))
# df_day.loc[(df_day['MTI'] > 0.1),'COLOR'] = 'G'
# df_day.loc[df_day['MTI'] < -0.1,'COLOR'] = 'R'
# df_day.loc[(df_day['MTI'] < 0.1) & (df_day['MTI'] > -0.1),'COLOR'] = 'Y'
df_day['SMA20'] = ta.sma(df_day['Close'], length=20)
df_day['SMA50'] = ta.sma(df_day['Close'], length=50)
df_day.dropna(inplace=True)
df_day['Color'] = 'Y'
df_day.loc[df_day['SMA20'] < df_day['SMA50'],'Color'] = 'R'
df_day.loc[(df_day['SMA20'] > df_day['SMA50']) & (df_day['Close'] > df_day['SMA20']) & (df_day['SMA20'] > df_day['SMA20'].shift(7)),'Color'] = 'G'
df_day.drop(columns=['SMA20', 'SMA50'], inplace=True)
# df_day.tail(50)

In [22]:
def chandelier_exit(df_day, length=22, mult=3.0, use_close=True):
        
    df = df_day.copy()
    
    # Calculate ATR
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # True Range calculation
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR using Wilder's smoothing (EMA with alpha = 1/length)
    atr = tr.ewm(alpha=1/length, adjust=False).mean()
    atr_mult = mult * atr
    
    # Calculate long stop
    if use_close:
        highest = close.rolling(window=length).max()
    else:
        highest = high.rolling(window=length).max()
    
    long_stop = highest - atr_mult
    
    # Apply long stop logic: can only move up or stay same
    long_stop_prev = long_stop.shift(1)
    close_prev = close.shift(1)
    
    for i in range(1, len(long_stop)):
        if pd.notna(long_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
            if close_prev.iloc[i] > long_stop_prev.iloc[i]:
                long_stop.iloc[i] = max(long_stop.iloc[i], long_stop_prev.iloc[i])
    
    # Calculate short stop
    # if use_close:
    #     lowest = close.rolling(window=length).min()
    # else:
    #     lowest = low.rolling(window=length).min()
    
    # short_stop = lowest + atr_mult
    
    # # Apply short stop logic: can only move down or stay same
    # short_stop_prev = short_stop.shift(1)
    
    # for i in range(1, len(short_stop)):
    #     if pd.notna(short_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
    #         if close_prev.iloc[i] < short_stop_prev.iloc[i]:
    #             short_stop.iloc[i] = min(short_stop.iloc[i], short_stop_prev.iloc[i])
    
    # Determine direction
    # direction = pd.Series(index=df.index, dtype=int)
    # direction.iloc[0] = 1  # Start with long
    
    # for i in range(1, len(direction)):
    #     if pd.notna(short_stop_prev.iloc[i]) and close.iloc[i] > short_stop_prev.iloc[i]:
    #         direction.iloc[i] = 1
    #     elif pd.notna(long_stop_prev.iloc[i]) and close.iloc[i] < long_stop_prev.iloc[i]:
    #         direction.iloc[i] = -1
    #     else:
    #         direction.iloc[i] = direction.iloc[i-1]
    
    # Generate signals
    # direction_prev = direction.shift(1)
    # buy_signal = (direction == 1) & (direction_prev == -1)
    # sell_signal = (direction == -1) & (direction_prev == 1)
    
    # Add indicator columns to the original dataframe
    df['Ch_exit'] = long_stop
    # df['short_stop'] = short_stop
    # df['direction'] = direction
    # df['buy_signal'] = buy_signal
    # df['sell_signal'] = sell_signal
    # df['atr'] = atr
    
    return df

df_day = chandelier_exit(df_day, use_close=False)
df_day.dropna(inplace=True)
# df_day.tail(50)

In [23]:
  
index_arr_day = df_day.index.to_numpy()
close_arr_day = df_day['Close'].to_numpy()
color_arr_day = df_day['Color'].to_numpy()
ch_exit_arr_day = df_day['Ch_exit'].to_numpy()

entry_arr_day = np.full(len(index_arr_day), False)
exit_arr_day = np.full(len(index_arr_day), False)
price_arr_day = np.full(len(index_arr_day), np.nan)

def backtest(df_day):

    trade_is_open = False
    close_price = np.nan

    for i in range(1, len(index_arr_day)):
        if not trade_is_open and color_arr_day[i] == 'G':
            trade_is_open = True
            entry_arr_day[i] = True
            close_price = ch_exit_arr_day[i]
            price_arr_day[i] = close_arr_day[i]
            print(f"CH = {ch_exit_arr_day[i]}")
            print(f"CL = {close_arr_day[i]}")
        elif trade_is_open and close_arr_day[i] < close_price:
            trade_is_open = False
            exit_arr_day[i] = True
            close_price = np.nan
            price_arr_day[i] = close_arr_day[i]
        else:
                continue
        
    return df_day

df_day = backtest(df_day)

CH = 26.657844952862632
CL = 27.62
CH = 25.42801638622121
CL = 26.77


In [24]:
pf = vbt.Portfolio.from_signals(
entries = entry_arr_day,
exits = exit_arr_day,
price = price_arr_day,
open = df_day["Open"],
close = close_arr_day,
size = config['Trade']['size'],
size_type = config['Trade']['size_type'],
init_cash = config['Initial_cash'],
freq = '1d'
)

In [25]:
pf.stats()

Start                         2014-04-14 00:00:00
End                           2025-12-16 00:00:00
Period                         2938 days 00:00:00
Start Value                               10000.0
End Value                                10279.93
Total Return [%]                           2.7993
Benchmark Return [%]                   1059.05838
Max Gross Exposure [%]                   3.143405
Total Fees Paid                               0.0
Max Drawdown [%]                         0.664038
Max Drawdown Duration           546 days 00:00:00
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                             280.96
Win Rate [%]                                  0.0
Best Trade [%]                          -3.729182
Worst Trade [%]                         -3.729182
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                    -3.729182


In [26]:
pf.trades.records_readable.tail(50)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,Open,1.0,2014-06-05,27.62,0.0,2014-10-13,26.59,0.0,-1.03,-0.037292,Long,Closed,0
1,1,Open,1.0,2015-02-10,26.77,0.0,2025-12-16,307.73,0.0,280.96,10.495331,Long,Open,1


In [27]:
from savetopdf import save_backtesting_results_to_pdf
file_path = config['Data_filename_day'].split('.')[0]
file_path= (f"{file_path}_{config['Days']}")
save_backtesting_results_to_pdf(pf, file_path)

CSV file with statistics 'data/GOOG_day_FMP_0_stats.pdf' was created!
